In [ ]:
import pandas as pd
import numpy as np

import time
from datetime import datetime, timedelta
import pytz

In [ ]:
# Deal with some poor datatime formatting.
# Some hours extend to 24:00 instead of 0:00 - 23:59
def standardize_datetime(date_str):
    
    # Check format of the strings:
    fmt = '%Y-%m-%d %H:%M:%S' if '-' in date_str else '%m/%d/%Y %H:%M'
    
    # Newer years denote the change of daylight savings time
    date_str = date_str.strip(' DST')
    
    # Strip decimal seconds
    date_str = date_str.split('.')[0]
    
    if '24:00' in date_str:
        #print(f'orig: {date_str}')
        date_str = date_str.replace('24:00', '23:00')
        new_dt = datetime.strptime(date_str, fmt)
        new_dt = new_dt+timedelta(hours=1)
        #print(f'final: {new_dt}')
    else:
        new_dt = datetime.strptime(date_str, fmt)
    
    return new_dt

# Set timezone and convert to UTC
# default timezone for TEXAS
def set_timezone(dt, is_dt_dst, zone='US/Central'):
    #print(dt)
    timezone = pytz.timezone(zone)
    new_dt = timezone.localize(dt, is_dst=is_dt_dst)
    new_dt = new_dt.astimezone(pytz.utc)
    #print(new_dt)
    return new_dt
    



def check_is_dst(df, dt_col='Hour_End'):
    
    is_dt_dst = False # All TX years start with January (not daylight savings time)
    
    dts = []
    for idx in df.index:
        as_str = df.loc[idx, dt_col]
        
        dts.append(standardize_datetime(as_str))
        
    df[dt_col] = dts
    # Need to round before making daylight savings comparisons
    df[dt_col] = pd.to_datetime(df[dt_col]).dt.round('H')
    
    prev = ''
    for idx in df.index:
        as_dt = df.loc[idx, dt_col]
        
        # Get prev set to work below
        if prev == '':
            prev = as_dt-timedelta(hours=1)
        
        # Find daylight savings time boundaries
        # Identical values are during Fall's fall back (exiting daylight savings)
        if as_dt == prev:
            print(f"\n -- two identicals at: {idx} with values {prev}")
            is_dt_dst = False
        # Skipping values is spring forward (entering daylight savings)
        elif as_dt != prev+timedelta(hours=1):
            print(f"\n -- dst: {idx} with values {prev}, {as_dt}")
            is_dt_dst = True
        prev = as_dt
        
        final_dt = set_timezone(as_dt, is_dt_dst, 'US/Central')
        df.loc[idx, dt_col] = final_dt
        
    return df
        
        
first = True
prep_mem = True
for YEAR in range(2002, 2020):
    if prep_mem:
        continue
    print(YEAR)
    TYPE = 'xls' if YEAR <= 2015 else 'xlsx'
    df = pd.read_excel(f'/Users/truggles/Downloads/ERCOT_DEMAND/{YEAR}_ERCOT_Hourly_Load_Data.{TYPE}',
                      dtype={'Hour_End': str})
    #if YEAR <= 2016:
    #    df['Hour_End'] = pd.to_datetime(df['Hour_End']).dt.round('H')
    #    check_is_dst(df, 'Hour_End')
    #else:
    
    df = check_is_dst(df, 'Hour_End')
        
    #print(df.head())
    print(f' - {len(df.index)}')
    
    if first:
        master = df
        master_cols = list(master.columns)
    else:
        if master_cols != list(df.columns):
            print(master_cols)
            print(list(df.columns))
            print("Yikes")
        master = master.append(df)
        print(f' - {len(master.index)}')
    first = False

if not prep_mem:
    master.to_csv('tmp3.csv', index=False)

    

In [ ]:
# Add MEM date/time columns and select the target demand region
def prep_mem(f_name, tgt_dem_col):
    
    df = pd.read_csv(f_name)
    df['Hour_End'] = pd.to_datetime(df['Hour_End'])
    print(df.dtypes)
    
    years, months, days, hours = [], [], [], []
    for idx in df.index:
        dt = df.loc[idx, 'Hour_End'] + timedelta(hours=-1)
        years.append(dt.year)
        months.append(dt.month)
        days.append(dt.day)
        hours.append(dt.hour + 1)
    
    new_df = pd.DataFrame({'year': years,
                           'month': months,
                           'day': days,
                           'hour': hours,
                           'demand (MW)': df[tgt_dem_col]})
    
    new_df.to_csv('new_tmp2.csv', index=False)
    return

prep_mem('tmp3.csv', 'ERCOT')
    

In [ ]:
import matplotlib.pyplot as plt

df = pd.read_csv('tmp3.csv')
df['Hour_End'] = pd.to_datetime(df['Hour_End'])

fig, ax = plt.subplots()
ax.plot(df['ERCOT'])
plt.show()